In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


import sys
import time
import json
import pymongo
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import Geohash

In [2]:
def dataprocess(iter):
    new_list = []
    ids = []
    flag = 0
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["assignment_taskc"]
    static_data = db["static_data"]
    print("Function called here mc")
    datas = iter.collect()
    for data in datas:
        #data = data.replace("\'","\"")
        #data = json.loads(data)
        #new_list.append(json.loads(data[1]))
        #print(json.loads(data[1]))
        my_data = json.loads(data[1])
        my_data = my_data.replace("'","\"")
        my_data = json.loads(my_data)
#         print(type(my_data))
#         print(my_data)
        new_list.append(my_data)
    
#     print("New list is ",(new_list))
    
    for elements in new_list:
        ids.append(elements["sender_id"])
        #print(Geohash.encode(elements["latitude"],elements["longitude"],precision = 5))
    
    print("Ids are ",ids)
    if len(set(ids)) == 1 and "P01" in ids:
        for elements in new_list:
            static_data.insert_one(elements)
            

    #if we receive data from only the two sources
    if len(set(ids)) == 2:
        climate_terra = ['P01','P02']
        climate_aqua = ['P01','P03']
        
        if((set(ids) == set(climate_terra)) or (set(ids) == set(climate_aqua))):
            #check locations hash and if same, join and insert
            #if not same, discard hotspot, insert climate
            parent = []
            child = []
            for index in range(len(new_list)):
                if new_list[index]["sender_id"] == "P01":
                    parent.append(new_list[index])
                else:
                    child.append(new_list[index])
            #print("Parent is ", parent)
            #print("child is ", child)
            
            for index1 in range(len(parent)):
                flag = 0
                my_parent = parent[index1]
                my_parent["hotspot"] = []
                hash1 = Geohash.encode(parent[index1]["latitude"],parent[index1]["longitude"],precision = 2)
                for index2 in range(len(child)):
                    hash2 = Geohash.encode(child[index2]["latitude"],child[index2]["longitude"],precision = 2)
                    if hash1 == hash2:
                        my_parent["hotspot"].append(child[index2])
                        #print("MERGE WILL TAKE PLACE")
                #print("PARENT IS ", my_parent)
                static_data.insert_one(my_parent)
    
    #now we will deal with with the scenario when we receive the dat a from all three sources 
    
    if len(set(ids))== 3:
        p1_data = []
        p2_data = []
        p3_data = []
        
        for index in range(len(new_list)):
            if new_list[index]["sender_id"]== "P01":
                p1_data.append(new_list[index])
            elif new_list[index]["sender_id"]== "P02":
                p2_data.append(new_list[index])
            else:
                p3_data.append(new_list[index])
            
        print("P1 DATA IS", p1_data)
        print("P2 DATA IS", p2_data)
        print("P3 DATA IS", p3_data)
        
        
        p3_index = []
        combined_data = []
        for index1 in range(len(p2_data)):
            p2 = p2_data[index1]
            flag = 0
            counter = 1
            hash1 = Geohash.encode(p2_data[index1]["latitude"],p2_data[index1]["longitude"],precision = 2)
            for index2 in range(len(p3_data)):
                hash2 = Geohash.encode(p3_data[index2]["latitude"],p3_data[index2]["longitude"],precision = 2)
                p3 = p3_data[index2]
                if hash1 == hash2:
                    print("HERE P2 AND P3 WILL BE MERGED")
                    print("BEFORE COMBINING IS", p2)
                    p2["surface_temperature_celcius"] = (counter * p2["surface_temperature_celcius"] + p3["surface_temperature_celcius"])//(counter+1)
                    p2["confidence"] = (counter*p2["confidence"] + p3["confidence"])//(counter+1)
                    p3_index.append(index2)
                    print("COMBINED DATA IS ", p2)
                    counter = counter + 1
            combined_data.append(p2)
        
        for index2 in range(len(p3_data)):
            if index2 not in p3_index:
                combined_data.append(p3_data[index2])
        
        print("FINAL COMBINED LIST IS", combined_data)
        
        for index1 in range(len(p1_data)):
                flag = 0
                my_parent = p1_data[index1]
                my_parent["hotspot"] = []
                hash1 = Geohash.encode(p1_data[index1]["latitude"],p1_data[index1]["longitude"],precision = 2)
                for index2 in range(len(combined_data)):
                    hash2 = Geohash.encode(combined_data[index2]["latitude"],combined_data[index2]["longitude"],precision = 2)
                    if hash1 == hash2:
                        print("MATCH FOUND BETWEEN P! AND COMBINED DATA")
                        my_parent["hotspot"].append(combined_data[index2])
                print("THE FINAL DATA TO BE STORED IN THE DB", my_parent)
        
                    
                        
                        
                    
                
            
                                     
n_secs = 10
topic = "taskc"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'taskc-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary


#ines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(dataprocess))
lines = kafkaStream.foreachRDD(dataprocess)


# lines = kafkaStream.map(lambda v: json.loads(v[1]))
# dataprocess(lines)

#show the thing 
#lines.map(lambda x: 'Data is : %s\n' %x).pprint()



ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
ssc.stop(stopSparkContext=True,stopGraceFully=True)


Function called here mc
Ids are  []
Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03', 'P02']
P1 DATA IS [{'longitude': 143.0203, 'windspeed_knots': 9.4, 'max_wind_speed': 14.0, 'timestamp': '02:00:21', 'sender_id': 'P01', 'latitude': -37.3583, 'air_temperature_celcius': 13, 'relative_humidity': 43.3, 'precipitation ': ' 0.00G'}, {'longitude': 149.27700000000002, 'windspeed_knots': 9.3, 'max_wind_speed': 13.0, 'timestamp': '02:00:26', 'sender_id': 'P01', 'latitude': -37.611, 'air_temperature_celcius': 18, 'relative_humidity': 49.4, 'precipitation ': ' 0.00I'}]
P2 DATA IS [{'surface_temperature_celcius': 51.0, 'timestamp': '02:00:27', 'sender_id': 'P02', 'confidence': 77.0, 'longitude': 141.849, 'latitude': -36.9455}]
P3 DATA IS [{'surface_temperature_celcius': 76.0, 'timestamp': '02:00:22', 'sender_id': 'P03', 'confidence': 95.0, 'longitude': 143.5186, 'latitude': -36.8681}, {'surface_temperature_celcius': 65.0, 'timestamp': '02:00:27', 'sender_id': 'P03', 'confidence': 100.

Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03']
Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03']
Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03', 'P02']
P1 DATA IS [{'longitude': 149.325, 'windspeed_knots': 9.3, 'max_wind_speed': 12.0, 'timestamp': '02:02:12', 'sender_id': 'P01', 'latitude': -37.6, 'air_temperature_celcius': 16, 'relative_humidity': 48.1, 'precipitation ': ' 0.00G'}, {'longitude': 143.2419, 'windspeed_knots': 9.1, 'max_wind_speed': 15.0, 'timestamp': '02:02:17', 'sender_id': 'P01', 'latitude': -37.4661, 'air_temperature_celcius': 14, 'relative_humidity': 48.2, 'precipitation ': ' 0.00G'}]
P2 DATA IS [{'surface_temperature_celcius': 54.0, 'timestamp': '02:02:17', 'sender_id': 'P02', 'confidence': 64.0, 'longitude': 142.063, 'latitude': -35.723}]
P3 DATA IS [{'surface_temperature_celcius': 49.0, 'timestamp': '02:02:12', 'sender_id': 'P03', 'confidence': 75.0, 'longitude': 141.5953, 'latitude': -36.433}, {'surface_temperature_celciu

Function called here mc
Ids are  ['P01', 'P03', 'P02', 'P01', 'P03']
P1 DATA IS [{'longitude': 144.35399999999998, 'windspeed_knots': 5.9, 'max_wind_speed': 13.0, 'timestamp': '02:02:52', 'sender_id': 'P01', 'latitude': -37.945, 'air_temperature_celcius': 24, 'relative_humidity': 55.6, 'precipitation ': ' 0.00I'}, {'longitude': 144.174, 'windspeed_knots': 11.9, 'max_wind_speed': 18.1, 'timestamp': '02:02:57', 'sender_id': 'P01', 'latitude': -37.864000000000004, 'air_temperature_celcius': 28, 'relative_humidity': 58.4, 'precipitation ': ' 0.04G'}]
P2 DATA IS [{'surface_temperature_celcius': 40.0, 'timestamp': '02:02:53', 'sender_id': 'P02', 'confidence': 59.0, 'longitude': 143.7445, 'latitude': -36.304}]
P3 DATA IS [{'surface_temperature_celcius': 42.0, 'timestamp': '02:02:52', 'sender_id': 'P03', 'confidence': 89.0, 'longitude': 147.1227, 'latitude': -36.7309}, {'surface_temperature_celcius': 46.0, 'timestamp': '02:02:57', 'sender_id': 'P03', 'confidence': 71.0, 'longitude': 142.1944, 

Function called here mc
Ids are  ['P02', 'P01', 'P03', 'P01', 'P03']
P1 DATA IS [{'longitude': 143.451, 'windspeed_knots': 6.4, 'max_wind_speed': 12.0, 'timestamp': '02:03:52', 'sender_id': 'P01', 'latitude': -37.614000000000004, 'air_temperature_celcius': 19, 'relative_humidity': 60.2, 'precipitation ': ' 0.00I'}, {'longitude': 142.7354, 'windspeed_knots': 12.9, 'max_wind_speed': 19.0, 'timestamp': '02:03:57', 'sender_id': 'P01', 'latitude': -36.7084, 'air_temperature_celcius': 13, 'relative_humidity': 44.1, 'precipitation ': ' 0.02G'}]
P2 DATA IS [{'surface_temperature_celcius': 46.0, 'timestamp': '02:03:50', 'sender_id': 'P02', 'confidence': 80.0, 'longitude': 148.143, 'latitude': -37.466}]
P3 DATA IS [{'surface_temperature_celcius': 71.0, 'timestamp': '02:03:52', 'sender_id': 'P03', 'confidence': 92.0, 'longitude': 142.8567, 'latitude': -36.663000000000004}, {'surface_temperature_celcius': 38.0, 'timestamp': '02:03:57', 'sender_id': 'P03', 'confidence': 76.0, 'longitude': 143.1971,

Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03']
Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03', 'P02']
P1 DATA IS [{'longitude': 142.6076, 'windspeed_knots': 5.5, 'max_wind_speed': 12.0, 'timestamp': '02:04:32', 'sender_id': 'P01', 'latitude': -36.5775, 'air_temperature_celcius': 19, 'relative_humidity': 58.7, 'precipitation ': ' 0.28G'}, {'longitude': 149.29399999999998, 'windspeed_knots': 6.8, 'max_wind_speed': 12.0, 'timestamp': '02:04:37', 'sender_id': 'P01', 'latitude': -37.62, 'air_temperature_celcius': 21, 'relative_humidity': 60.4, 'precipitation ': ' 0.00I'}]
P2 DATA IS [{'surface_temperature_celcius': 58.0, 'timestamp': '02:04:37', 'sender_id': 'P02', 'confidence': 84.0, 'longitude': 144.6536, 'latitude': -37.671}]
P3 DATA IS [{'surface_temperature_celcius': 106.0, 'timestamp': '02:04:32', 'sender_id': 'P03', 'confidence': 100.0, 'longitude': 147.0336, 'latitude': -37.4375}, {'surface_temperature_celcius': 85.0, 'timestamp': '02:04:37', 'sender_id'

Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03']
Function called here mc
Ids are  ['P02', 'P01', 'P03', 'P01', 'P03']
P1 DATA IS [{'longitude': 142.3635, 'windspeed_knots': 8.5, 'max_wind_speed': 15.0, 'timestamp': '02:05:42', 'sender_id': 'P01', 'latitude': -36.0966, 'air_temperature_celcius': 22, 'relative_humidity': 57.0, 'precipitation ': ' 0.00G'}, {'longitude': 142.031, 'windspeed_knots': 10.1, 'max_wind_speed': 15.0, 'timestamp': '02:05:47', 'sender_id': 'P01', 'latitude': -34.965, 'air_temperature_celcius': 20, 'relative_humidity': 62.6, 'precipitation ': ' 0.00I'}]
P2 DATA IS [{'surface_temperature_celcius': 45.0, 'timestamp': '02:05:41', 'sender_id': 'P02', 'confidence': 70.0, 'longitude': 142.4985, 'latitude': -36.6348}]
P3 DATA IS [{'surface_temperature_celcius': 48.0, 'timestamp': '02:05:42', 'sender_id': 'P03', 'confidence': 65.0, 'longitude': 141.9795, 'latitude': -36.8946}, {'surface_temperature_celcius': 44.0, 'timestamp': '02:05:47', 'sender_id': 'P03', '

Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03', 'P02']
P1 DATA IS [{'longitude': 142.3405, 'windspeed_knots': 6.3, 'max_wind_speed': 13.0, 'timestamp': '02:06:32', 'sender_id': 'P01', 'latitude': -36.1002, 'air_temperature_celcius': 23, 'relative_humidity': 49.6, 'precipitation ': ' 0.00I'}, {'longitude': 148.05200000000002, 'windspeed_knots': 6.5, 'max_wind_speed': 12.0, 'timestamp': '02:06:37', 'sender_id': 'P01', 'latitude': -37.365, 'air_temperature_celcius': 11, 'relative_humidity': 43.8, 'precipitation ': ' 0.02G'}]
P2 DATA IS [{'surface_temperature_celcius': 62.0, 'timestamp': '02:06:39', 'sender_id': 'P02', 'confidence': 87.0, 'longitude': 145.7544, 'latitude': -36.1757}]
P3 DATA IS [{'surface_temperature_celcius': 58.0, 'timestamp': '02:06:32', 'sender_id': 'P03', 'confidence': 84.0, 'longitude': 144.6536, 'latitude': -37.671}, {'surface_temperature_celcius': 112.0, 'timestamp': '02:06:37', 'sender_id': 'P03', 'confidence': 100.0, 'longitude': 142.141, 'latitude'

Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03']
Function called here mc
Ids are  ['P02', 'P01', 'P03', 'P01', 'P03']
P1 DATA IS [{'longitude': 141.6716, 'windspeed_knots': 8.0, 'max_wind_speed': 15.0, 'timestamp': '02:07:52', 'sender_id': 'P01', 'latitude': -36.279, 'air_temperature_celcius': 13, 'relative_humidity': 42.5, 'precipitation ': ' 0.00I'}, {'longitude': 143.5051, 'windspeed_knots': 14.4, 'max_wind_speed': 23.9, 'timestamp': '02:07:57', 'sender_id': 'P01', 'latitude': -36.9817, 'air_temperature_celcius': 14, 'relative_humidity': 44.0, 'precipitation ': ' 0.16G'}]
P2 DATA IS [{'surface_temperature_celcius': 54.0, 'timestamp': '02:07:52', 'sender_id': 'P02', 'confidence': 80.0, 'longitude': 142.2025, 'latitude': -34.2695}]
P3 DATA IS [{'surface_temperature_celcius': 39.0, 'timestamp': '02:07:53', 'sender_id': 'P03', 'confidence': 66.0, 'longitude': 147.1254, 'latitude': -36.7508}, {'surface_temperature_celcius': 60.0, 'timestamp': '02:07:58', 'sender_id': 'P03', 

Function called here mc
Ids are  ['P01', 'P03', 'P01', 'P03']
Function called here mc
Ids are  ['P02', 'P01', 'P03', 'P01', 'P03']
P1 DATA IS [{'longitude': 148.091, 'windspeed_knots': 6.7, 'max_wind_speed': 15.0, 'timestamp': '02:09:03', 'sender_id': 'P01', 'latitude': -37.471, 'air_temperature_celcius': 9, 'relative_humidity': 40.8, 'precipitation ': ' 0.01G'}, {'longitude': 142.7354, 'windspeed_knots': 12.9, 'max_wind_speed': 19.0, 'timestamp': '02:09:08', 'sender_id': 'P01', 'latitude': -36.7084, 'air_temperature_celcius': 13, 'relative_humidity': 44.1, 'precipitation ': ' 0.02G'}]
P2 DATA IS [{'surface_temperature_celcius': 59.0, 'timestamp': '02:09:02', 'sender_id': 'P02', 'confidence': 85.0, 'longitude': 143.8866, 'latitude': -35.7523}]
P3 DATA IS [{'surface_temperature_celcius': 51.0, 'timestamp': '02:09:03', 'sender_id': 'P03', 'confidence': 77.0, 'longitude': 143.7532, 'latitude': -38.0171}, {'surface_temperature_celcius': 61.0, 'timestamp': '02:09:08', 'sender_id': 'P03', 'c

Function called here mc
Ids are  ['P01', 'P03', 'P02', 'P01', 'P03']
P1 DATA IS [{'longitude': 149.32299999999998, 'windspeed_knots': 7.9, 'max_wind_speed': 11.1, 'timestamp': '02:09:53', 'sender_id': 'P01', 'latitude': -37.623000000000005, 'air_temperature_celcius': 19, 'relative_humidity': 56.8, 'precipitation ': ' 0.00I'}, {'longitude': 149.29399999999998, 'windspeed_knots': 6.8, 'max_wind_speed': 12.0, 'timestamp': '02:09:58', 'sender_id': 'P01', 'latitude': -37.62, 'air_temperature_celcius': 21, 'relative_humidity': 60.4, 'precipitation ': ' 0.00I'}]
P2 DATA IS [{'surface_temperature_celcius': 73.0, 'timestamp': '02:09:57', 'sender_id': 'P02', 'confidence': 93.0, 'longitude': 143.0466, 'latitude': -38.1693}]
P3 DATA IS [{'surface_temperature_celcius': 81.0, 'timestamp': '02:09:53', 'sender_id': 'P03', 'confidence': 97.0, 'longitude': 141.6875, 'latitude': -36.4211}, {'surface_temperature_celcius': 81.0, 'timestamp': '02:09:58', 'sender_id': 'P03', 'confidence': 97.0, 'longitude': 